### 1 - (A) , (B)



In [1]:
! pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from datasets import load_dataset
ds = load_dataset("KaungHtetCho/Harry_Potter_LSTM")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

(…)in/Harry Potter 1 - Sorcerer's Stone.txt:   0%|          | 0.00/443k [00:00<?, ?B/s]

(…)/Harry Potter 2 - Chamber of Secrets.txt:   0%|          | 0.00/490k [00:00<?, ?B/s]

(…)y Potter 3 - The Prisoner Of Azkaban.txt:   0%|          | 0.00/615k [00:00<?, ?B/s]

...

(…)rry Potter 6 - The Half Blood Prince.txt:   0%|          | 0.00/986k [00:00<?, ?B/s]

(…)est/Harry Potter 7 - Deathly Hollows.txt:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/57435 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5897 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6589 [00:00<?, ? examples/s]

In [4]:
train = ds['test'][0]['text']
print(train)
print(type(train))

Harry Potter and the Deathly Hallows By J. K. Rowling
<class 'str'>


In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# for reproducibility
torch.manual_seed(100)

In [6]:
# Dictionary
sample_sentences = [train]
char_set = list(set(''.join(sample_sentences)))
dic = {c: i for i, c in enumerate(char_set)}

# Parameters
dic_size = len(dic)
input_size = dic_size
hidden_size = dic_size * 2
output_size = dic_size

# Dataset setting
input_batch = []
target_batch = []

for sentence in sample_sentences:
  x_data = [dic[c] for c in sentence[:-1]]
  x_one_hot = [np.eye(dic_size)[x] for x in x_data]
  y_data = [dic[c] for c in sentence[1:]]
  input_batch.append(x_one_hot)
  target_batch.append(y_data)

# To torch tensors
X = torch.FloatTensor(np.array(input_batch))
Y = torch.LongTensor(np.array(target_batch))

print(X.shape)
print(Y.shape)

torch.Size([1, 52, 23])
torch.Size([1, 52])


In [7]:
# Model
class Custom_RNN(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, layers):
    super(Custom_RNN, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers)
    self.fc = torch.nn.Linear(hidden_dim, output_dim, bias=True)

  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

learning_rate = 0.05
training_epochs = 100
model_rnn = Custom_RNN(input_size, hidden_size, output_size, 2)

In [8]:
# define cost/loss & optimizer
criterion = nn.CrossEntropyLoss()    # Softmax
optimizer = optim.Adam(model_rnn.parameters(), lr=learning_rate)

# train
for epoch in range(training_epochs):
  optimizer.zero_grad()
  outputs = model_rnn(X)
  loss = criterion(outputs.reshape(-1, dic_size), Y.reshape(-1))
  loss.backward()
  optimizer.step()
  if epoch % 10 == 9:
    print('epoch: ',epoch, 'loss: ', loss.item())

result = outputs.data.numpy().argmax(axis=2)
for sentence in result:
  print(''.join([char_set[c] for c in np.squeeze(sentence)]))


epoch:  9 loss:  1.008851170539856
epoch:  19 loss:  0.937755823135376
epoch:  29 loss:  0.9267765879631042
epoch:  39 loss:  0.9205278754234314
epoch:  49 loss:  0.9188652038574219
epoch:  59 loss:  0.9176897406578064
epoch:  69 loss:  0.9175123572349548
epoch:  79 loss:  0.9173024892807007
epoch:  89 loss:  0.9172506332397461
epoch:  99 loss:  0.9172519445419312
anyy Bowhh yBng Bhe Be nhel Banllwl By B. B. Bowllng


2-(A) (B) (C) (D)

In [9]:
from datasets import load_dataset

ds_movie = load_dataset("sepidmnorozy/Korean_sentiment")

train.csv:   0%|          | 0.00/3.29M [00:00<?, ?B/s]

dev.csv:   0%|          | 0.00/127k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/239k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/36000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1333 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2667 [00:00<?, ? examples/s]

In [10]:
ds_movie['test'][0:3]['text']

['이 영화는 아름다운 영상미로 쓰레기같은 내용을 감추고 멋진 영화로 위장을 하고있다. 나는 이 영화가 만들다가 만 영화라고 생각한다.',
 '워메 ㅋㅋ 아이언맨이 여기나오는줄은 ㅋㅋ반전이네요',
 '여자 진짜 개귀엽다 ㅋㅋㅋㅋ']

In [11]:
import evaluate

accuracy = evaluate.load('accuracy')

import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [12]:
model1_name = 'jzonthemtn/distilbert-imdb'
model2_name = 'lvwerra/distilbert-imdb'


from transformers import AutoModelForSequenceClassification
model1 = AutoModelForSequenceClassification.from_pretrained(model1_name)
model2 = AutoModelForSequenceClassification.from_pretrained(model2_name)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [13]:
from transformers import pipeline
classifier1 = pipeline('sentiment-analysis', model=model1_name, device=0)
classifier2 = pipeline('sentiment-analysis', model=model2_name, device=0)

tokenizer_config.json:   0%|          | 0.00/360 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [14]:
average_accuracy_1 = 0.0 # 모델 1의 평균 정확도
average_accuracy_2 = 0.0 # 모델 2의 평균 정확도

for repeat in range(0, 3):
  test_list = ds_movie['test']
  test_sample_size = 100
  rng = np.random.default_rng()
  random_indices = rng.choice(len(test_list), size=test_sample_size, replace=False)
  sampled_data = test_list.select(random_indices)
  model_info = ""
  for i in range(1, 3):
    if i == 1:
      model = model1
      classifier = classifier1
      model_info = "model1"

    elif i==2:
      model = model2
      classifier = classifier2
      model_info = "model2"

    test_predictions = classifier(sampled_data['text'], max_length=512, truncation=True)
    predicted_labels = [label['label'] for label in test_predictions]
    mapped_labels = [model.config.label2id[label] for label in predicted_labels]
    reference_labels = sampled_data['label']
    print(f"Model: {model_info}")
    print(accuracy.compute(predictions=mapped_labels, references=reference_labels))
    print(reference_labels)
    print(mapped_labels)
    print()
    if i == 1:
      average_accuracy_1 += (accuracy.compute(predictions=mapped_labels, references=reference_labels)['accuracy'] / 3)
    elif i==2:
      average_accuracy_2 += (accuracy.compute(predictions=mapped_labels, references=reference_labels)['accuracy'] / 3)

Model: model1
{'accuracy': 0.52}
[0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0]
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]

Model: model2
{'accuracy': 0.51}
[0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [16]:
print(f'모델 1의 평균 성능: {average_accuracy_1}')
print(f'모델 2의 평균 성능: {average_accuracy_2}')

if average_accuracy_1 > average_accuracy_2:
  print('모델 1이 모델 2 보다 평균적으로 우수합니다.')
elif average_accuracy_1 < average_accuracy_2:
  print('모델 2가 모델 1 보다 평균적으로 우수합니다.')
else:
  print('모델 1와 모델 2의 성능은 평균적으로 동일합니다.')

모델 1의 평균 성능: 0.5166666666666666
모델 2의 평균 성능: 0.5133333333333334
모델 1이 모델 2 보다 평균적으로 우수합니다.
